In [49]:
from bs4 import BeautifulSoup
import requests
import random
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as bs
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")
service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

def extract_sitejabber():
    name_list= []
    title_list = []
    date_list = []
    ratings_list = []
    reviews_list = []
    for i in range(1,9):
        driver.get(f"https://www.sitejabber.com/reviews/lazada.sg?page={i}#reviews")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        interested_section = soup.find('div',attrs={"class": "url-reviews__list"})
        review_list = interested_section.find_all('div', id=lambda x: x and x.startswith('ReviewRow'))
        for review in review_list:
            date = review.find('div',attrs={"class":"review__date"}).text.strip()
            date_list.append(date)
            name = review.find('span',attrs={"class":"review__author__name__link"}).text.strip().replace("$nbsp", "")
            name_list.append(name)
            title = review.find('span',attrs={"class":"review__title__text"}).text.strip()
            title_list.append(title)
            review_text = review.find('p').text.strip("\n")
            reviews_list.append(review_text)
            rating = review.find('div', attrs={'class':"stars stars stars-container stars-container--normal"})['data-rating']
            ratings_list.append(rating)
            
    return {'date':date_list,'name':name_list,'rating':ratings_list,'title':title_list,'review':reviews_list}

In [50]:
reviews = extract_sitejabber()

In [61]:
df = pd.DataFrame(reviews)
df['date'] = pd.to_datetime(df['date'].str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    189 non-null    datetime64[ns]
 1   name    189 non-null    object        
 2   rating  189 non-null    object        
 3   title   189 non-null    object        
 4   review  189 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 7.5+ KB


In [62]:
df.to_excel('sitejabber_reviews.xlsx',index=False)